In [1]:
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

class DataGen:
    def __init__(self) -> None:
        self.batch_size=16
        self.img_h = 331
        self.img_w = 331
        self.train_data_dir=r'./train/'
        self.valid_data_dir=r'./train/'

    def data_processing(self):
        train_datagen = ImageDataGenerator(
                        rescale=1./255,           # 将像素值缩放到 [0,1] 范围内
                        rotation_range=15,        # 随机旋转图像的角度范围
                        horizontal_flip=True
        )
        val_datagen = ImageDataGenerator(rescale=1./255)

        train_generate = train_datagen.flow_from_directory(
            self.train_data_dir,
            target_size=(self.img_h, self.img_w),
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=True
        )    

        val_generate = val_datagen.flow_from_directory(
            self.valid_data_dir,
            target_size=(self.img_h, self.img_w),
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=True
        )
        return train_generate, val_generate

In [4]:
class Model_Train:
    """
    model_weight_base_url: https://storage.googleapis.com/tensorflow
    
    """
    def __init__(self, model_save_url) -> None:
        self.num_classes = 2
        self.train_generate, self.val_generate = DataGen().data_processing()
        self.model_save_url = model_save_url

    def Inception_v3_model(self):
        model_inception_v3 = keras.applications.resnet.ResNet50(include_top=False)
        for layer in model_inception_v3.layers:
            layer.trainable=False
        model = keras.Sequential()
        model.add(model_inception_v3)
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dense(1024, activation = 'relu'))
        model.add(layers.Dense(self.num_classes, activation = 'sigmoid'))
        model.compile(optimizer=keras.optimizers.Adam(lr=0.0005), loss = 'categorical_crossentropy', metrics = ['acc'])
        return model

    def train(self):
        model = self.Inception_v3_model()
        history = model.fit_generator(
            self.train_generate,
            steps_per_epoch = 400//16,
            epochs = 500,
            validation_data = self.val_generate,
            validation_steps = 400//16
        )
        # 模型保存
        model.save(self.model_save_url, overwrite=True)

if __name__ == "__main__":
    Model_Train(r'./result_model_test_1121.h5').train()

Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
94773248/94765736 [==============================] - 8s 0us/step
Epoch 1/500
25/25 [==============================] - 1087s 43s/step - loss: 0.7404 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/500
22/25 [=========================>....] - ETA: 1:35 - loss: 0.6931 - acc: 0.5028

KeyboardInterrupt: 

In [7]:
weights='/Users/huanghuixiong/Documents/Sammy/pretrain_model/tensorflow_model_weigth/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'.encode('utf-8')
weights.decode('utf-8')


'/Users/huanghuixiong/Documents/Sammy/pretrain_model/tensorflow_model_weigth/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'